# Load libraries 

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# load data

In [6]:
data_dir = '/workspaces/PRODIGY_ML_03/dogs-vs-cats-sample'  # Directory containing the sample dataset
img_size = 64  
data = []
labels = []


In [7]:
# Load and process images
for filename in os.listdir(data_dir):
    try:
        # Construct full file path
        file_path = os.path.join(data_dir, filename)
        
        # Read the image in grayscale
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        
        if img is not None:
            # Resize the image
            img = cv2.resize(img, (img_size, img_size))
            
            # Flatten the image
            img_flatten = img.flatten()
            
            # Append image data and label
            data.append(img_flatten)
            labels.append(0 if 'dog' in filename else 1)
        else:
            print(f"Image at {file_path} not found or can't be read.")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

In [8]:
# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

print(f"Loaded {len(data)} images successfully.")

Loaded 100 images successfully.
